<img src="../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

## Running QX Examples Using IBM Data Science Experience

This example is copied from the original [tutorial4developer.ipynb](https://github.com/IBM/qiskit-sdk-py/blob/master/tutorial/sections/tutorial4developer.ipynb). It is tweaked (see the first code cell), so that it can be directly run using [IBM Data Science Experience](datascience.ibm.com) (IBM DSX). If you want to run other QX notebooks in IBM DSX, you can simply copy the contents from first *code cell* and paste at the beginning of your other notebook.

***

### About IBM Data Science Experience

IBM Data Science Experience supports the data scientist community to learn, create, and collaborate. 

If you just starting with Qiskit and looking for a webhosted Jupyter notebook environment, this is a good option. It is fairly easy to get your QX notebook running in the Jupyter notebook environment provided by IBM DSX. 

**This example illustrates how to tweak your notebook so that it is ready to be run using IBM DSX.**

* In the first code cell:
   * Update the value of **`QX_API_token`**
   * When you run the cell, you will see this prompt: ***"Do you want to proceed with this Python dependency installation? (y/n)"*** . Here, give an appropriate response (**y** for yes, **n** for no)!

* To run any other Quantum experience notebooks in IBM  Data Science Experience ...
  * Copy the first *code cell* of this notebook and paste it as the first *code cell* of your '*other*' QX notebook, then run that other notebook. 
  

### Credits and Contributions
1. The list of the contributors to the main QISKit SDK tutorial can be found in this notebook: [tutorial4developer.ipynb](https://github.com/IBM/qiskit-sdk-py/blob/master/tutorial/sections/tutorial4developer.ipynb).
2. The code for preparing the example to be run on IBM DSX uses this  [public gist](https://gist.github.com/dtmcclure/a1299fc99d12f5dc37ad6400fb8bbf2a) (See github user [dtmcclure](https://github.com/dtmcclure) ) as a reference and does further modifications. 
3. Ninad Sathaye (notebook customization for running it on IBM Data Science Experience)
***

For more information about how to use the IBM Q experience (QX), consult the [tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0), or check out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).


## QISKit SDK Tutorial

This tutorial aims to explain how to use the QISKit SDK from a developer's point of view. We review the steps it takes to install and start to use the SDK tools.

QISKit is a Python software development kit (SDK) that you can use to create your quantum computing programs based on circuits defined through the [OpenQASM 2.0 specification](https://github.com/IBM/qiskit-openqasm), compile them, and execute them on several backends (Real Quantum Processors online, Simulators online, and Simulators on local). For the online backends, QISKit uses our [python API connector](https://github.com/IBM/qiskit-api-py) to the [IBM Q experience project](http://quantumexperience.ng.bluemix.net/).

In addition to this tutorial, we have other tutorials that introduce you to more complex concepts directly related to quantum computing.

More examples:
- Familiarize yourself with the important concepts of [superposition and entanglement](superposition_and_entanglement.ipynb).
- Go beyond and explore a bit more in-depth in [entanglement revisited](entanglement_revisited.ipynb).



## Basic Concept

The basic concept of our quantum program is an array of quantum circuits. The program workflow consists of three stages: [Build](#sectionB), [Compile](#sectionC), and [Run](#sectionR). Build allows you to make different quantum circuits that represent the problem you are solving; Compile allows you to rewrite them to run on different backends (simulators/real chips of different [quantum volumes](http://ibm.biz/qiskit-quantum-volume), sizes, fidelity, etc); and Run launches the jobs.  After the jobs have been run, the data is collected. There are methods for putting this data together, depending on the program. This either gives you the answer you wanted or allows you to make a better program for the next instance.


In [ ]:
# This cell does some preparatory work to run this Jupyter notebook in IBM Data Science 
# Experience (datascience.ibm.com). All the remaining cells (except some 
# imports in the next cell) remain the same as in the original example. 

import sys
import os

print("sys.version_info:", sys.version_info)
if sys.version_info < (3, 0):
    RELOAD_MODULE = reload
else:
    # -----------------------------------------------------------
    # importlib.reload is used to reload the Qconfig 
    # Fixes a problem where Qconfig.APItoken is not getting the 
    # updated values upon reexecution.
    # -----------------------------------------------------------
    import importlib
    RELOAD_MODULE = importlib.reload

# We need visibility to Qconfig.py module and qiskit package. Add qiskit-sdk-py to your sys.path
sys.path.append("./")

# -----------------------------------------------------------------------------------------------
# Move the older version of qiskit-sdk-py directory (if any) as "OLD.qiskit-sdk-py"
# NOTE: If you want to retrieve any data from "OLD.qiskit-sdk-py" NOW WOULD BE THE TIME!!
# Because, if you re-run this cell later, it will just overwrite "OLD.qiskit-sdk-py". 
# Alternatively, you can modify the code below to change the default behavior. 
# -----------------------------------------------------------------------------------------------
if os.path.isdir("./qiskit-sdk-py"):   
    print("qiskit-sdk-py directory exists. Moving it as OLD.qiskit-sdk-py")    
    if os.path.isdir("./OLD.qiskit-sdk-py"):
        !rm -rf OLD.qiskit-sdk-py               
    !mv -T qiskit-sdk-py OLD.qiskit-sdk-py

# Clone the qiskit-sdk-py repository
!git clone https://github.com/IBM/qiskit-sdk-py    
os.chdir('./qiskit-sdk-py')             
print("About to install the Python dependencies for qiskit-sdk-py.")
print("See qiskit-sdk-py/requires.txt to see the dependencies and their versions to be installed")        
!pip install --upgrade -r requires.txt

# -----------------------------------------------------------------------------------------------
# There appears to be a bug in IBMQuantumExperience v1.8 which results in a 'KeyError' 
# while getting the results back. Temporarily, rollback to v1.6
# -----------------------------------------------------------------------------------------------
!pip install IBMQuantumExperience==1.6

# -----------------------------------------------------------------------------------------------
# Log on to your QX account and get your API token. You can get it from the Accounts page 
# (the top-right icon on QX website) Paste this API token below.BE SURE TO ENCLOSE IN QUOTES ("")
# -----------------------------------------------------------------------------------------------
QX_API_TOKEN = "PUT_YOUR_API_TOKEN_HERE" 

# Update the Qconfig.py file so as to connect to the QX API.
!cp qiskit-sdk-py/tutorial/Qconfig.py.default Qconfig.py
# Replace the token string with the one from above! Do not change this line.
!sed -e 's/#APItoken = "PUT_YOUR_API_TOKEN_HERE"/APItoken = "$QX_API_TOKEN"/' -i qiskit-sdk-py/Qconfig.py

# Now import the required modules
from qiskit import QuantumProgram
import Qconfig

try:
    # Reload Qconfig module again to make sure the APItoken is updated
    RELOAD_MODULE(Qconfig)
except NameError as e:
    print("Name error: ", e.args)
except:
    print("unable to reload the module, continuing the execution..")
    
print("Qconfig.APItoken = ", Qconfig.APItoken)

# ------------------------------------------------------------------------------------------------
# DO NOT CHANGE THE FOLLOWING LINE. It makes an assertion that you have changed the value of 
#  QX_API_TOKEN above (although it does not check if the token is valid)
# ------------------------------------------------------------------------------------------------
assert Qconfig.APItoken != "PUT_YOUR_API_TOKEN_HERE"

The basic elements needed for your first program are the QuantumProgram, a Circuit, a Quantum Register, and a Classical Register.

In [ ]:
# Creating Programs
# create your first QuantumProgram object instance.
Q_program = QuantumProgram()

# Creating Registers
# create your first Quantum Register called "qr" with 2 qubits 
qr = Q_program.create_quantum_registers("qr", 2)
# create your first Classical Register  called "cr" with 2 bits
cr = Q_program.create_classical_registers("cr", 2)

# Creating Circuits
# create your first Quantum Circuit called "qc" involving your Quantum Register "qr"
# and your Classical Register "cr"
qc = Q_program.create_circuit("qc", ["qr"], ["cr"])

Another option for creating your QuantumProgram instance is to define a dictionary with all the necessary components of your program.

In [ ]:
Q_SPECS = {
    "circuits": [{
        "name": "Circuit",
        "quantum_registers": [{
            "name": "qr",
            "size": 4
        }],
        "classical_registers": [{
            "name": "cr",
            "size": 4
        }]}],
}

The required element for a Program is a "circuits" array. Within "circuits", the required field is "name"; it can have several Quantum Registers and Classical Registers. Every register must have a name and the number of each element (qubits or bits).

After that, you can use this dictionary definition as the specs of one QuantumProgram object to initialize it.

In [ ]:
Q_program = QuantumProgram(specs=Q_SPECS)

You can also get every component from your new Q_program to use.

In [ ]:
# Get the components.

# get the circuit by Name
circuit = Q_program.get_circuit("Circuit")

# get the Quantum Register by Name
quantum_r = Q_program.get_quantum_registers("qr")

# get the Classical Register by Name
classical_r = Q_program.get_classical_registers('cr')

### Building your program: Add Gates to your Circuit
After you create the circuit with its registers, you can add gates to manipulate the registers. Below is a list of the gates you can use in the QX.

You can find extensive information about these gates and how use them in our [Quantum Experience User Guide](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=71972f437b08e12d1f465a8857f4514c&pageIndex=2). 

In [ ]:
# H (Hadamard) gate to the qubit 0 in the Quantum Register "qr" 
circuit.h(quantum_r[0])

# Pauli X gate to the qubit 1 in the Quantum Register "qr" 
circuit.x(quantum_r[1])

# Pauli Y gate to the qubit 2 in the Quantum Register "qr" 
circuit.y(quantum_r[2])

# Pauli Z gate to the qubit 3 in the Quantum Register "qr" 
circuit.z(quantum_r[3])

# CNOT (Controlled-NOT) gate from qubit 0 to the Qbit 2
circuit.cx(quantum_r[0], quantum_r[2])

# add a barrier to your circuit
circuit.barrier()

# first physical gate: u1(lambda) to qubit 0
circuit.u1(0.3, quantum_r[0])

# second physical gate: u2(phi,lambda) to qubit 1
circuit.u2(0.3, 0.2, quantum_r[1])

# second physical gate: u3(theta,phi,lambda) to qubit 2
circuit.u3(0.3, 0.2, 0.1, quantum_r[2])

# S Phase gate to qubit 0
circuit.s(quantum_r[0])

# T Phase gate to qubit 1
circuit.t(quantum_r[1])

# identity gate to qubit 1
circuit.iden(quantum_r[1])

# Note: "if" is not implemented in the local simulator right now,
#       so we comment it out here. You can uncomment it and
#       run in the online simulator if you'd like.

# Classical if, from qubit2 gate Z to classical bit 1
# circuit.z(quantum_r[2]).c_if(classical_r, 0)

# measure gate from the qubit 0 to classical bit 0
circuit.measure(quantum_r[0], classical_r[0])


### Extract QASM

You can obtain a QASM representation of your code.

In [ ]:
# QASM from a program

QASM_source = Q_program.get_qasm("Circuit")

print(QASM_source)

### Compile and Run or Execute <a id='sectionC'></a>

In [ ]:
device = 'ibmqx_qasm_simulator' # Backend to execute your program, in this case it is the online simulator
circuits = ["Circuit"]  # Group of circuits to execute

Q_program.set_api(Qconfig.APItoken, Qconfig.config["url"]) # set the APIToken and API url

In [ ]:
Q_program.compile(circuits, device) # Compile your program

# Run your program in the device and check the execution result every 2 seconds 
result = Q_program.run(wait=2, timeout=240)

print(result)

When you run a program, the possible results will be:

```
JOB_STATUS = {
    inProgress: 'RUNNING',
    errorOnCreate: 'ERROR_CREATING_JOB',
    errorExecuting: 'ERROR_RUNNING_JOB',
    completed: 'COMPLETED'
  };
```

The *run()* command waits until the job either times out, returns an error message, or completes successfully. 

In [ ]:
Q_program.get_counts("Circuit")

In addition to getting the number of times each output was seen, you can get the compiled QASM.
For this simulation, the compiled circuit is not much different from the input circuit. Each single-qubit gate has been expressed as a u1, u2, or u3 gate.

In [ ]:
compiled_qasm = Q_program.get_compiled_qasm("Circuit")

print(compiled_qasm)

You can use *execute()* to combine the compile and run in a single step.

In [ ]:
Q_program.execute(circuits, device, wait=2, timeout=240)

#### Compile Parameters
Q_program.compile(circuits, device="simulator", shots=1024, max_credits=3, basis_gates=None, coupling_map=None, seed=None)
     - circuits: Array of circuits to compile
     - device: Backend 
        ["simulator",               # online default simulator links to ibmqx_qasm_simulator
         "real",                    # online default real chip links to ibmqx2
         "ibmqx_qasm_simulator"     # qasm simulator 
         "ibmqx2",                  # online real chip with 5 qubits
         "ibmqx3",                  # online real chip with 16 qubits
         "local_unitary_simulator", # local unitary simulator 
         "local_qasm_simulator"]    # local simulator 
     - shots: Number of shots, only for real chips and qasm simulators
     - max_credits: Maximum number of the credits to spend in the executions. If the executions cost more than your available credits, the job is aborted
     - basis_gates: the base gates by default are: u1, u2, u3, cx, id
     - coupling_map: Object that represents the physical/topological layout of a chip.
     - seed: For the qasm simulator if you want to set the initial seed. 
#### Run Parameters
Q_program.run(wait=5, timeout=60)
     - wait: Time to wait before checking if the execution is COMPLETED.
     - timeout: Timeout of the execution.
#### Execute Parameters 
*Execute has the combined parameters of compile and run.*

Q_program.execute(circuits, device, shots=1024,
                max_credits=3, basis_gates=None, wait=5, timeout=60, basis_gates=None, coupling_map=None,)

### Execute on a Real Device<a id='sectionR'></a>

In [ ]:
device = 'ibmqx2'   # Backend where you execute your program; in this case, on the Real Quantum Chip online 
circuits = ["Circuit"]   # Group of circuits to execute
shots = 1024           # Number of shots to run the program (experiment); maximum is 8192 shots.
max_credits = 3          # Maximum number of credits to spend on executions. 

result = Q_program.execute(circuits, device, shots, max_credits=3, wait=10, timeout=240)

### Result
You can access the result via the function *get_counts("circuit_name")*. By default, the last device is used, but you can be more specific by using *get_counts("circuit_name", device="device_name")*.

In [ ]:
Q_program.get_counts("Circuit")

### Execute on a local simulator

In [ ]:
Q_program.compile(circuits, "local_qasm_simulator") # Compile your program

# Run your program in the device and check the execution result every 2 seconds 
result = Q_program.run(wait=2, timeout=240)

Q_program.get_counts("Circuit")